# Laden der Daten
In diesem Notebook werden die Daten eingelesen, welche direkt über die URL zugängig sind. Die Geodaten für das Weg- und Velonetz <br>
in Zürich müssen manuell auf der Open Data Seite runtergeladen und dann über einen CSV-Import geladen werden, nachdem die Datei Entzipped wurde.<br>
Grund hierfür ist die Auswahl des Ausschnitts, in welchem das Velonetz angezeigt werden soll. <br>
Dieser muss auf der Seite ausgewählt werden und es wird dann ein Downlaodlink für 5 Minuten zur Verfügung gestellt.


## Verwendete Links/Quellen

In [6]:
url1 = "https://www.hev-schweiz.ch/vermieten/nebenkostenabrechnungen/sonnenscheindauer"
url2 = "https://www.hev-schweiz.ch/vermieten/statistiken/mietpreise/durchschnittliche-mietpreise"
url3 = "https://opendata.swiss/de/dataset/median-strompreis-per-kanton"

# Installation notwendiger Bibliotheken

In [7]:
!pip install --upgrade pip
!pip install beautifulsoup4
!pip install openpyxl

# Module importieren

In [8]:
from bs4 import BeautifulSoup as Soup
import urllib.request
import openpyxl
import pandas as pd
from pathlib import Path

# Funktionen

In [9]:
def tableDataText(table):
    def rowgetDataText(tr, coltag='td'):
        return [td.get_text(strip=True) for td in tr.find_all(coltag)]
    rows = []
    trs = table.find_all('tr')
    headerow = rowgetDataText(trs[0], 'th')
    if headerow:
        rows.append(headerow)
        trs = trs[1:]
    for tr in trs:
        rows.append(rowgetDataText(tr, 'td') )
    return rows

In [17]:
html = urllib.request.urlopen(url1).read()
soup = Soup(html, parser='lxml')
htmltables = soup.find_all('table', { 'class' : 'contenttable acc_content ce-table' })

all_lists = []

year = 2024

for table in htmltables:
    list_table = tableDataText(table)
    all_lists.append(list_table)

df = pd.DataFrame(all_lists)

df.columns = df.iloc[0]
df = df[1:]

'''
If we want to persist Sonnenscheindauer.csv beyond a colab session we could also
try to save it into a google drive folder using for example:

from google.colab import drive
drive.mount('/content/drive')
filepath = Path('/content/drive/My Drive/Sonnenscheindauer.csv')
'''
#Filepath by Kevin. Both files are to be found in "/content" folder. Probably default by colab.
filepath = Path('/content/Sonnenscheindauer.csv')

filepath.parent.mkdir(parents=True, exist_ok=True)
df.to_csv(filepath, index=False)

for table in all_lists:
    print(table)

TypeError: unhashable type: 'list'